In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:10pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="6" color="red">Ch 15. 데이터베이스연동</font><b>

# 1. SQLite 데이터베이스 연결
- SQLite 데이터베이스는 별도의 DBMS 없이, SQL문 이용해 DB 액세스할 수 있도록 만든 간단한 디스크 기반 DB
- C라이브러리
- SQLite는 프로토타입 생성 시 사용

- 프로젝트 : 분석 → 설계 → 구현 → 테스트 → 고객 배포 → 유지보
            → 프로토타입(시제품)         완제품
- DB Browser for SLQLITE' 설치 및 sqlite 패키지 load;

## 1.1. SQLite browser 설치 및 sqlite 패키지 load

In [3]:
import sqlite3
sqlite3.sqlite_version # 버전

'3.40.1'

## 1.2. 데이터베이스 연결
```
(1) 데이터베이스 연결
(2) SQL 전송 객체 생성(cursor)
(3) SQL 전송(cursor.execute() 사용) & 결과 받기
(4) cursor 해제 & 데이터베이스 연결 객체 해제
```

In [4]:
# (1) DB 연결 : SQLite로 연결 시 DB / 없으면 빈 파일로 
conn = sqlite3.connect('data/ch15_example.db')
conn

In [5]:
# (2) SQL 전송 객체 생성(cursor)
cursor = conn.cursor()
cursor

In [9]:
# (3) SQL 전송(cursor.execute() 사용) & 결과 받기
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT, 
        AGE INT,
        EMAIL TEXT)
''')

In [8]:
# cursor.execute('DROP TABLE HERE')

In [10]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동', 20, 'h@hong.com'))
print('수행결과 행수: ', cursor.rowcount)

수행결과 행수:  1


In [11]:
sql = "INSERT INTO MEMBER VALUES ('김길동', 30, 'k@k.com')"
cursor.execute(sql)
print('수행결과 행수: ', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동', 25, 'l@l.com')")
print('수행결과 행수: ', cursor.rowcount)

수행결과 행수:  1
수행결과 행수:  1


In [12]:
conn.commit() # (반) conn.rollback() DML은 conn으로

In [13]:
# select 전송 : cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER")

In [15]:
# insert, update, delete문 실행 결과 확인 : cursor.rowcount
# select문 실행 결과 확인 
    # fetchall() : 결과를 모두 받을 때(튜플리스트)
    # fetchone() : 결과를 한 행씩 받을 때(튜플)
    # fetchmany(n) : 결과를 n행 받을 때(튜플리스트)
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com'), ('김길동', 30, 'k@k.com'), ('이길동', 25, 'l@l.com')]


In [16]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch될 수 없음

[]


In [17]:
# 한꺼번에 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@hong.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'k@k.com')]

In [18]:
for member in members:
    print(member)

('홍길동', 20, 'h@hong.com')
('이길동', 25, 'l@l.com')
('김길동', 30, 'k@k.com')


In [23]:
# 하나씩 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)
members

데이터 끝


[('홍길동', 20, 'h@hong.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'k@k.com')]

In [24]:
# 최상위 n개 행 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@hong.com')
('이길동', 25, 'l@l.com')


In [25]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [33]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): # 튜플매개변수를 받아 Member형 객체를 return
    return Member(row[0], row[1], row[2])

In [34]:
member = to_member('홍길동', 20, 'h@h.com')
print(member)

홍길동	20	h@h.com


In [35]:
cursor.execute('SELECT * FROM MEMBER')
member_list = [] #sql문 수행한 결과를 담을 객체 list
members = cursor.fetchall() #튜플리스트
#print(members)
for member in members:
    member_list.append(to_member(*member))

In [36]:
for member in member_list:
    print(member)

홍길동	20	h@hong.com
김길동	30	k@k.com
이길동	25	l@l.com


In [37]:
# (4) cursor 해제 & 데이터베이스 연결 객체 해제
cursor.close()
conn.close()

## 1.3. SQL 구문에 파라미터 사용하기
- qmark : DB에 따라 사용 불가한 경우 있음
- named(추천)

In [38]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('홍길동', '김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@hong.com'), ('김길동', 30, 'k@k.com')]

In [41]:
# 파라미터 사용하기 : qmark 방법
name1 = input('검색할 이름 1: ')
name2 = input('검색할 이름 2: ')
#cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('"+name1+"', '"+name2+"')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?, ?)", (name1, name2))
cursor.fetchall()

검색할 이름 1: 홍길동
검색할 이름 2: 고길동


[('홍길동', 20, 'h@hong.com')]